# SuAVE LeNet CNN Model Generator
For a simple Python implementation see https://www.pyimagesearch.com/2016/08/01/lenet-convolutional-neural-network-in-python/

The model was originally described in http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf

In [9]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

### Import all packages (this might take a few seconds)

In [2]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from imutils import paths

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K
# import local lenet.py file describing the LeNet implementation with RELU activation functions
from lenet import LeNet

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

Using TensorFlow backend.


## Initializing the number of times the model will loop and its batch size for learning

In [3]:
epochCount = {
    '25 Iterations': 25,
    '50 Iterations': 50,
    '75 Iterations': 75
}

def f(epoch_count):
    return epoch_count

epochNum = interact(f, epoch_count=epochCount)

interactive(children=(Dropdown(description='epoch_count', options={'75 Iterations': 75, '25 Iterations': 25, '…

In [4]:
batchS = {
    '32 Batch Size': 32,
    '64 Batch Size': 64,
    '128 Batch Size': 128
}

def f(batch_size):
    return batch_size

batchNum = interact(f, batch_size=batchS)

interactive(children=(Dropdown(description='batch_size', options={'32 Batch Size': 32, '64 Batch Size': 64, '1…

In [5]:
# init the number of epochs to train for, init learning rate and batch size
EPOCHS = epochNum.widget.result
INIT_LR = 1e-3
BS = batchNum.widget.result
# init the image suffix, yset, and image list
suffix = '.jpg'
img_list = []
yset = []
# create labels list and 2 dicts for 2 way mapping
labels = []
# key = label value = number
label_yval = dict()
# key = number value = label
yval_label = dict()

## Run and choose the column name that coresponds with the column label to be predicted

In [6]:
# use csv file to grab images/labels
csv_path = "../../temp_csvs/" + csv_file
df = pd.read_csv(csv_path)

#generate image path
#lower_case_csv = csv_file.lower()
#lower_case_csv = lower_case_csv.split(user + "_")


img_path = "../../images/" + dzc_file.split("/")[-1].split(".")[0]



# Choose column of label for prediction
toPredict = list(df.columns.values)

pred_menu = {}
for i in range(0, len(toPredict)):
    pred_menu[toPredict[i]] = toPredict[i]

def f(predictions_menu):
    return predictions_menu
# choose which label for predictions
out2 = interact(f, predictions_menu=pred_menu)

interactive(children=(Dropdown(description='predictions_menu', options={'Taxonomic Name': 'Taxonomic Name', 'T…